In [1]:
# !git clone https://github.com/pthengtr/kcw-analytics.git

In [2]:
# !cd /content/kcw-analytics && git pull origin main

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
import os
import sys

if "google.colab" in sys.modules:
    # Running in Colab
    from google.colab import drive
    drive.mount("/content/drive")
    
    BASE_FOLDER = "/content/drive/Shareddrives"
    BASE_FOLDER_GIT = "/content"
else:
    # Running in local Jupyter
    BASE_FOLDER = r"G:\Shared drives"
    BASE_FOLDER_GIT = r"C:\Users\Windows 11\Notebook"

print("Using folder:", BASE_FOLDER)

Using folder: G:\Shared drives


In [5]:
folder = f"{BASE_FOLDER}/KCW-Data/kcw_analytics/01_raw"

In [6]:
import os
import pandas as pd

data = {}

for file in os.listdir(folder):
    if file.endswith(".csv"):
        path = os.path.join(folder, file)
        data[file] = pd.read_csv(
            path,
            dtype={
              "BCODE": "string",
              "ITEMNO": "string",
              "BILLNO": "string",
            },
            encoding="utf-8-sig",
            low_memory=False   # stops chunk guessing
        )
        print(f"Loaded: {file} -> {data[file].shape}")

Loaded: raw_inventory_hq_2024.csv -> (4983, 8)
Loaded: raw_syp_simas_sales_bills.csv -> (12366, 49)
Loaded: raw_hq_pidet_purchase_lines.csv -> (153764, 41)
Loaded: raw_syp_sidet_sales_lines.csv -> (36551, 38)
Loaded: raw_syp_pimas_purchase_bills.csv -> (2919, 49)
Loaded: raw_hq_simas_sales_bills.csv -> (275969, 49)
Loaded: raw_hq_pimas_purchase_bills.csv -> (50222, 49)
Loaded: raw_hq_sidet_sales_lines.csv -> (733097, 38)
Loaded: raw_syp_pidet_purchase_lines.csv -> (27266, 41)
Loaded: raw_hq_icmas_products.csv -> (114922, 94)
Loaded: raw_hq_pvmas_notes_vouchers.csv -> (13730, 32)


In [7]:
import pandas as pd
import numpy as np

def build_bill_summary_by_taxic(
    df: pd.DataFrame,
    *,
    billno_col: str = "BILLNO",
    billdate_col: str = "BILLDATE",
    detail_col: str = "DETAIL",
    amount_col: str = "AMOUNT",
    taxic_col: str = "TAXIC",
    tax_rate: float = 0.07,
    tax_id_value: str = "0000000000000",
    taxic_yes: str = "Y",
):
    """
    VAT logic:
      TAXIC == Y : AMOUNT includes VAT
      TAXIC == N : AMOUNT excludes VAT
    """

    out = df.copy()
    out[billno_col] = out[billno_col].astype("string")
    out[amount_col] = pd.to_numeric(out[amount_col], errors="coerce").fillna(0)

    # normalize TAXIC
    if taxic_col not in out.columns:
        out[taxic_col] = ""

    out[taxic_col] = out[taxic_col].astype("string").str.strip().str.upper()

    # ===== pick DETAIL from highest AMOUNT row =====
    idx_max_amt = out.groupby(billno_col)[amount_col].idxmax()
    detail_pick = (
        out.loc[idx_max_amt, [billno_col, detail_col]]
        .set_index(billno_col)[detail_col]
    )

    # ===== group totals =====
    totals = (
        out.groupby(billno_col, as_index=False)
        .agg(
            TOTAL_AMOUNT=(amount_col, "sum"),
            BILLDATE=(billdate_col, "first"),
            TAXIC=(taxic_col, "first"),
        )
    )

    totals[detail_col] = totals[billno_col].map(detail_pick)

    # ===== VAT calculation based on TAXIC =====
    divisor = 1 + tax_rate
    is_vat_inclusive = totals["TAXIC"].eq(taxic_yes)

    totals["BEFORE_VAT"] = np.where(
        is_vat_inclusive,
        (totals["TOTAL_AMOUNT"] / divisor).round(2),   # inclusive case
        totals["TOTAL_AMOUNT"].round(2)               # exclusive case
    )

    totals["VAT_AMOUNT"] = np.where(
        is_vat_inclusive,
        (totals["TOTAL_AMOUNT"] - totals["BEFORE_VAT"]).round(2),
        (totals["TOTAL_AMOUNT"] * tax_rate).round(2)
    )

    # optional: total including VAT (very useful downstream)
    totals["TOTAL_INCL_VAT"] = (totals["BEFORE_VAT"] + totals["VAT_AMOUNT"]).round(2)

    # TAX ID
    totals["TAX_ID"] = str(tax_id_value).zfill(13)[:13]

    # SEQ
    totals = totals.sort_values(billno_col).reset_index(drop=True)
    totals["SEQ"] = np.arange(1, len(totals) + 1)

    return totals

def map_simas_bill_fields(
    df: pd.DataFrame,
    df_simas: pd.DataFrame,
    *,
    billno_col: str = "BILLNO",
    fields: tuple = ("DEDUCT", "TAX", "AFTERTAX"),
    copy: bool = True,
    verbose: bool = True,
):
    """
    Join DEDUCT, TAX, AFTERTAX from df_simas into df using BILLNO.
    """

    if billno_col not in df.columns:
        raise ValueError(f"{billno_col} not found in df")

    if billno_col not in df_simas.columns:
        raise ValueError(f"{billno_col} not found in df_simas")

    # --- normalize join keys (VERY important for legacy POS)
    left = df.copy()
    right = df_simas.copy()

    left["_JOIN_KEY"] = left[billno_col].astype("string").str.strip().str.upper()
    right["_JOIN_KEY"] = right[billno_col].astype("string").str.strip().str.upper()

    # --- build lookup table (avoid duplicate explosion)
    cols = ["_JOIN_KEY"] + [c for c in fields if c in right.columns]

    lookup = (
        right[cols]
        .drop_duplicates(subset=["_JOIN_KEY"], keep="first")
    )

    # --- merge
    result = left.merge(
        lookup,
        on="_JOIN_KEY",
        how="left"
    ).drop(columns=["_JOIN_KEY"])

    if copy:
        result = result.copy()

    if verbose:
        matched = result[fields[0]].notna().sum()
        print(f"[map_simas_bill_fields] matched rows: {matched:,}/{len(result):,}")

    return result

def filter_year_month(df, year, month, date_col="BILLDATE"):
    return df[pd.to_datetime(df[date_col]).dt.to_period("M") == f"{year}-{month:02d}"]

In [8]:
import pandas as pd

# 1) Config: doc type prefixes per source
BILLTYPE_RULES = {
    "hq": {
        "TD":  ("TD",),
        "TAD": ("TAD",),
        "TR":  ("TR",),
        "CN":  ("CN",),
    },
    "syp": {
        "TD":  ("3TD",),
        "TAD": ("3TAD",),
        "TR":  ("3TR",),
        "CN":  ("3CN",),
    },
}

def build_monthly_doc_summaries(
    df_sidet: pd.DataFrame,
    df_simas: pd.DataFrame,
    *,
    source: str,     # "hq" or "syp"
    year: int,
    month: int,
    billno_col: str = "BILLNO",
    date_col: str = "BILLDATE",
    verbose: bool = True,
):
    """
    Clean pipeline:
      1) filter month
      2) remove TF bills
      3) split TD/TAD/TR/CN
      4) build summary (TAXIC logic)
      5) map SIMAS fields
    """

    # ---- 1) filter month
    df_m = filter_year_month(df_sidet, year, month)

    # normalize billno once
    s = df_m[billno_col].astype("string").str.strip().str.upper()

    # ---- 2) remove TF bills
    df_m = df_m.loc[~s.str.startswith("TF", na=False)].copy()
    s = df_m[billno_col].astype("string").str.strip().str.upper()

    # ---- 3) prefix rules (simple, no config dict needed)
    if source == "hq":
        rules = {
            "TD":  ("TD",),
            "TAD": ("TAD",),
            "TR":  ("TR",),
            "CN":  ("CN",),
        }
    else:  # syp
        rules = {
            "TD":  ("3TD",),
            "TAD": ("3TAD",),
            "TR":  ("3TR",),
            "CN":  ("3CN",),
        }

    out = {}

    for doc_type, prefixes in rules.items():

        mask = s.str.startswith(prefixes, na=False)
        df_type = df_m.loc[mask].copy()

        if df_type.empty:
            out[doc_type] = df_type
            if verbose:
                print(f"[{source}] {doc_type}: 0 rows")
            continue

        # ---- 4) build summary (VAT logic by TAXIC)
        summ = build_bill_summary_by_taxic(
            df_type,
            billno_col=billno_col,
            billdate_col=date_col,
        )

        # ---- 5) map SIMAS fields
        summ = map_simas_bill_fields(
            summ,
            df_simas,
            billno_col=billno_col,
            fields=("DEDUCT", "TAX", "AFTERTAX"),
            verbose=verbose,
        )

        out[doc_type] = summ

        if verbose:
            print(f"[{source}] {doc_type}: {len(df_type):,} rows -> {len(summ):,} bills")

    return out

In [9]:
df_hq_sidet = data["raw_hq_sidet_sales_lines.csv"].copy()
df_syp_sidet = data["raw_syp_sidet_sales_lines.csv"].copy()

df_hq_simas = data["raw_hq_simas_sales_bills.csv"].copy()
df_syp_simas = data["raw_syp_simas_sales_bills.csv"].copy()

In [10]:
dt = pd.to_datetime(df_hq_sidet["BILLDATE"], errors="coerce")

YEAR  = dt.dt.year.max()
MONTH = dt[dt.dt.year == YEAR].dt.month.max()

print(YEAR, MONTH)

2026 2


In [11]:
hq_summaries = build_monthly_doc_summaries(
    df_hq_sidet, df_hq_simas,
    source="hq", year=YEAR, month=MONTH
)

syp_summaries = build_monthly_doc_summaries(
    df_syp_sidet, df_syp_simas,
    source="syp", year=YEAR, month=MONTH
)

# Access:
df_hq_td_summary  = hq_summaries["TD"]
df_hq_tad_summary = hq_summaries["TAD"]
df_hq_tr_summary  = hq_summaries["TR"]
df_hq_cn_summary  = hq_summaries["CN"]

df_syp_td_summary  = syp_summaries["TD"]
df_syp_tad_summary = syp_summaries["TAD"]
df_syp_tr_summary  = syp_summaries["TR"]
df_syp_cn_summary  = syp_summaries["CN"]

[map_simas_bill_fields] matched rows: 4/148
[hq] TD: 325 rows -> 148 bills
[map_simas_bill_fields] matched rows: 0/555
[hq] TAD: 609 rows -> 555 bills
[map_simas_bill_fields] matched rows: 0/30
[hq] TR: 71 rows -> 30 bills
[map_simas_bill_fields] matched rows: 41/41
[hq] CN: 44 rows -> 41 bills
[syp] TD: 0 rows
[syp] TAD: 0 rows
[map_simas_bill_fields] matched rows: 0/9
[syp] TR: 37 rows -> 9 bills
[syp] CN: 0 rows


In [12]:
import os

kcwdir = os.path.join(BASE_FOLDER, "KCW-Data")
print(kcwdir)

G:\Shared drives\KCW-Data


In [13]:
import os
from pathlib import Path

out_dir = os.path.join(
    kcwdir,
    "kcw_analytics",
    "04_outputs",
    "VAT_Sales",
    f"{YEAR}_{MONTH:02d}"
)
os.makedirs(out_dir, exist_ok=True)

exports = [
    (df_hq_td_summary,  "TD"),
    (df_hq_tad_summary, "TAD"),
    (df_hq_tr_summary,  "TR"),
    (df_hq_cn_summary,  "CN"),
    (df_syp_td_summary,  "3TD"),
    (df_syp_tad_summary, "3TAD"),
    (df_syp_tr_summary,  "3TR"),
    (df_syp_cn_summary,  "3CN"),
]

for df, name in exports:
    path = os.path.join(out_dir, f"{name}.csv")
    df.to_csv(path, index=False, encoding="utf-8-sig")
    print(f"Saved -> {path}")

Saved -> G:\Shared drives\KCW-Data\kcw_analytics\04_outputs\VAT_Sales\2026_02\TD.csv
Saved -> G:\Shared drives\KCW-Data\kcw_analytics\04_outputs\VAT_Sales\2026_02\TAD.csv
Saved -> G:\Shared drives\KCW-Data\kcw_analytics\04_outputs\VAT_Sales\2026_02\TR.csv
Saved -> G:\Shared drives\KCW-Data\kcw_analytics\04_outputs\VAT_Sales\2026_02\CN.csv
Saved -> G:\Shared drives\KCW-Data\kcw_analytics\04_outputs\VAT_Sales\2026_02\3TD.csv
Saved -> G:\Shared drives\KCW-Data\kcw_analytics\04_outputs\VAT_Sales\2026_02\3TAD.csv
Saved -> G:\Shared drives\KCW-Data\kcw_analytics\04_outputs\VAT_Sales\2026_02\3TR.csv
Saved -> G:\Shared drives\KCW-Data\kcw_analytics\04_outputs\VAT_Sales\2026_02\3CN.csv


In [14]:
df_pidet = data["raw_hq_pidet_purchase_lines.csv"].copy()
df_pimas = data["raw_hq_pimas_purchase_bills.csv"].copy()


In [15]:
def filter_year_month(df, year, month, date_col="BILLDATE"):
    d = df.copy()
    d[date_col] = pd.to_datetime(d[date_col], errors="coerce")
    return d[d[date_col].dt.to_period("M") == f"{year}-{month:02d}"]

In [16]:
df_pimas_m = filter_year_month(df_pimas, YEAR, MONTH)
df_pidet_m = filter_year_month(df_pidet, YEAR, MONTH)

In [17]:
# --- copy first (safe practice)
pidet_join = df_pidet_m.copy()
pimas_join = df_pimas_m.copy()

# --- normalize BILLNO (VERY important for old POS)
pidet_join["_JOIN_KEY"] = pidet_join["BILLNO"].astype("string").str.strip().str.upper()
pimas_join["_JOIN_KEY"] = pimas_join["BILLNO"].astype("string").str.strip().str.upper()

# --- build lookup from pimas (avoid duplicate explosion)
lookup = (
    pimas_join[["_JOIN_KEY", "BOOKNO", "TAX", "AFTERTAX"]]
    .drop_duplicates(subset=["_JOIN_KEY"], keep="first")
)

# --- LEFT JOIN (pidet is base)
pidet_join = pidet_join.merge(
    lookup,
    on="_JOIN_KEY",
    how="left"
).drop(columns=["_JOIN_KEY"])

print("rows after join:", len(pidet_join))
pidet_join.head()

rows after join: 2550


,ID,JOURMODE,JOURTYPE,JOURDATE,BILLTYPE,BILLDATE,BILLNO,LINE,ITEMNO,BCODE,...,PAID,SALEDATE,SALENO,SALEPRICE,ACCT_NO,CANCELED,DONE,BOOKNO,TAX,AFTERTAX
0,287444,2,PJ,2026-02-01,1,2026-02-01,0152.,10.0,1.0,12051971,...,N,NaN,NaN,NaN,NaN,N,N,1.0,0.0,9600.0
1,287445,2,PJ,2026-02-01,1,2026-02-01,0152.,20.0,2.0,12051621,...,N,NaN,NaN,NaN,NaN,N,N,1.0,0.0,9600.0
2,287635,2,PJ,2026-02-01,1,2026-02-01,0169.,10.0,1.0,12051545,...,N,NaN,NaN,NaN,NaN,N,N,1.0,0.0,2450.0
3,287636,2,PJ,2026-02-01,1,2026-02-01,0169.,20.0,2.0,30051472,...,N,NaN,NaN,NaN,NaN,N,N,1.0,0.0,2450.0
4,287793,2,PJ,2026-02-01,1,2026-02-01,0193.,10.0,1.0,18012060,...,N,NaN,NaN,NaN,NaN,N,N,1.0,0.0,12200.0


In [18]:
pidet_join = pidet_join[pidet_join["ISVAT"] == "Y"]

In [20]:
import pandas as pd
import numpy as np

def build_purchase_bill_summary_by_taxic(
    df: pd.DataFrame,
    *,
    billno_col: str = "BILLNO",
    billdate_col: str = "BILLDATE",
    detail_col: str = "DETAIL",
    amount_col: str = "AMOUNT",
    taxic_col: str = "TAXIC",
    bookno_col: str = "BOOKNO",
    tax_col: str = "TAX",
    aftertax_col: str = "AFTERTAX",
    tax_rate: float = 0.07,
    tax_id_value: str = "0000000000000",
    taxic_yes: str = "Y",
):
    """
    Purchase summary:
      - AMOUNT summed from pidet lines
      - KEEP BOOKNO, TAX, AFTERTAX from input (pimas)
      - VAT split based on TAXIC
    """

    out = df.copy()

    out[billno_col] = out[billno_col].astype("string").str.strip().str.upper()
    out[amount_col] = pd.to_numeric(out[amount_col], errors="coerce").fillna(0)

    if taxic_col not in out.columns:
        out[taxic_col] = ""

    out[taxic_col] = out[taxic_col].astype("string").str.strip().str.upper()

    # ===== pick DETAIL from highest AMOUNT row =====
    idx_max_amt = out.groupby(billno_col)[amount_col].idxmax()
    detail_pick = (
        out.loc[idx_max_amt, [billno_col, detail_col]]
        .set_index(billno_col)[detail_col]
    )

    # ===== group totals (KEEP BOOKNO / TAX / AFTERTAX) =====
    totals = (
        out.groupby(billno_col, as_index=False)
        .agg(
            TOTAL_AMOUNT=(amount_col, "sum"),
            BILLDATE=(billdate_col, "first"),
            TAXIC=(taxic_col, "first"),
            BOOKNO=(bookno_col, "first"),
            TAX=(tax_col, "first"),
            AFTERTAX=(aftertax_col, "first"),
        )
    )

    totals[detail_col] = totals[billno_col].map(detail_pick)

    # ===== VAT calculation based on TAXIC =====
    divisor = 1 + tax_rate
    is_vat_inclusive = totals["TAXIC"].eq(taxic_yes)

    totals["BEFORE_VAT"] = np.where(
        is_vat_inclusive,
        (totals["TOTAL_AMOUNT"] / divisor).round(2),
        totals["TOTAL_AMOUNT"].round(2)
    )

    totals["VAT_AMOUNT"] = np.where(
        is_vat_inclusive,
        (totals["TOTAL_AMOUNT"] - totals["BEFORE_VAT"]).round(2),
        (totals["TOTAL_AMOUNT"] * tax_rate).round(2)
    )

    totals["TOTAL_INCL_VAT"] = (totals["BEFORE_VAT"] + totals["VAT_AMOUNT"]).round(2)

    totals["TAX_ID"] = str(tax_id_value).zfill(13)[:13]

    totals = totals.sort_values(billno_col).reset_index(drop=True)
    totals["SEQ"] = np.arange(1, len(totals) + 1)

    return totals

In [21]:
purchase_summary = build_purchase_bill_summary_by_taxic(pidet_join)

,BILLNO,TOTAL_AMOUNT,BILLDATE,TAXIC,BOOKNO,TAX,AFTERTAX,DETAIL,BEFORE_VAT,VAT_AMOUNT,TOTAL_INCL_VAT,TAX_ID,SEQ
0,-1003861-2026,14336.45,2026-02-23,N,1.0,1003.55,15340.00,น้ำยาแอร์ R134a 134A ถังเล็ก 3K PACO,14336.45,1003.55,15340.00,0000000000000,1
1,-202602/00074,5800.00,2026-02-02,N,1.0,406.00,6206.00,"แขนลากผานคูโบต้ารุ่นดาม 35"" M5000-M6040 (RH ...",5800.00,406.00,6206.00,0000000000000,2
2,-202602/00078,3400.00,2026-02-02,N,1.0,238.00,3638.00,เฟืองบายศรี+เดือยหมูเพลาหน้า M5000-6040 8Tx1 ...,3400.00,238.00,3638.00,0000000000000,3
3,-202602/00082,5600.00,2026-02-02,N,1.0,392.00,5992.00,กระบอกช่วยยก กระบอกใหญ่(ฝาเกลียว) M8540-M9540...,5600.00,392.00,5992.00,0000000000000,4
4,-202602/00301,13425.00,2026-02-04,N,1.0,939.75,14364.75,แขนกลาง จัมโบ้ 6640 F/5000-6600 ย.1 DT,13425.00,939.75,14364.75,0000000000000,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
453,VV0025446,5370.00,2026-02-21,N,1.0,375.90,5745.90,คันส่งพวงมาลัยทั้งชุด ไซโคลน (K14) 333,5370.00,375.90,5745.90,0000000000000,454
454,VV0025447,1280.00,2026-02-21,N,1.0,89.60,1369.60,"ลูกหมากปีกนกล่าง T/T VIGO,REVO2W 333",1280.00,89.60,1369.60,0000000000000,455
455,VVA6900338,4107.01,2026-02-05,N,1.0,287.49,4394.50,แหวนลูกสูบ NPR 6D16-6D17,4107.01,287.49,4394.50,0000000000000,456
456,WC-690200828,270.00,2026-02-12,N,2.0,18.90,288.90,หม้อกรองเครื่อง BOBCAT K-FLO,270.00,18.90,288.90,0000000000000,457


In [22]:
import os

# your existing folder
out_dir = os.path.join(
    kcwdir,
    "kcw_analytics",
    "04_outputs",
    "VAT_Purchases",
    f"{YEAR}_{MONTH:02d}"
)
os.makedirs(out_dir, exist_ok=True)

# --- export loop
for bookno, g in purchase_summary.groupby("BOOKNO", dropna=False):

    # safe filename
    bookno_str = "UNKNOWN" if pd.isna(bookno) else str(bookno).strip()
    safe_bookno = "".join(c if c.isalnum() or c in ("-","_") else "_" for c in bookno_str)

    file_path =  os.path.join(out_dir, f"VAT_PURCHASE_{YEAR}_{MONTH:02d}_BOOK_{safe_bookno}.csv")

    g.to_csv(file_path, index=False, encoding="utf-8-sig")

    print("saved:", file_path)

saved: G:\Shared drives\KCW-Data\kcw_analytics\04_outputs\VAT_Purchases\2026_02\VAT_PURCHASE_2026_02_BOOK_1_0.csv
saved: G:\Shared drives\KCW-Data\kcw_analytics\04_outputs\VAT_Purchases\2026_02\VAT_PURCHASE_2026_02_BOOK_2_0.csv
saved: G:\Shared drives\KCW-Data\kcw_analytics\04_outputs\VAT_Purchases\2026_02\VAT_PURCHASE_2026_02_BOOK_5_0.csv
saved: G:\Shared drives\KCW-Data\kcw_analytics\04_outputs\VAT_Purchases\2026_02\VAT_PURCHASE_2026_02_BOOK_6_0.csv
saved: G:\Shared drives\KCW-Data\kcw_analytics\04_outputs\VAT_Purchases\2026_02\VAT_PURCHASE_2026_02_BOOK_UNKNOWN.csv
